In [11]:
cities = {
    "Boston": '42.331960, -71.020173',
    "Seattle": '47.620499, -122.350876',
    "Miami": '25.775163, -80.208615',
    "Cleveland": '41.478462, -81.679435',
    "Nashville": '36.171800, -86.785002'
}

api_key = 'cfc6c7845bf3bab98a881bc32987677a'

In [12]:
import datetime
import requests
url = 'https://api.forecast.io/forecast/'
#current_time = datetime.datetime.now().strftime("%s")
current_time = int((datetime.datetime.now() - datetime.datetime(1970,1,1)).total_seconds())
api_call = url + api_key + '/' + cities['Boston'] + ',' + str(current_time)
#current_time

r = requests.get(api_call)
r.json()['daily']['data'][0]

{u'apparentTemperatureMax': 51.09,
 u'apparentTemperatureMaxTime': 1450897200,
 u'apparentTemperatureMin': 44.07,
 u'apparentTemperatureMinTime': 1450918800,
 u'cloudCover': 0.69,
 u'dewPoint': 47.17,
 u'humidity': 0.94,
 u'icon': u'rain',
 u'moonPhase': 0.44,
 u'ozone': 261.1,
 u'precipIntensity': 0.0101,
 u'precipIntensityMax': 0.0688,
 u'precipIntensityMaxTime': 1450929600,
 u'precipProbability': 0.75,
 u'precipType': u'rain',
 u'pressure': 1021.81,
 u'summary': u'Rain starting in the evening.',
 u'sunriseTime': 1450872721,
 u'sunsetTime': 1450905396,
 u'temperatureMax': 51.09,
 u'temperatureMaxTime': 1450897200,
 u'temperatureMin': 46.06,
 u'temperatureMinTime': 1450872000,
 u'time': 1450846800,
 u'visibility': 5.84,
 u'windBearing': 85,
 u'windSpeed': 3.17}

In [14]:
import sqlite3 as lite

con = lite.connect('weather.db')
cur = con.cursor()

with con:
    cur.execute("DROP TABLE IF EXISTS daily_temp")
    cur.execute(
        """CREATE TABLE daily_temp (
                query_time INT,
                Boston REAL,
                Seattle REAL,
                Miami REAL,
                Cleveland REAL,
                Nashville REAL);"""
    )

In [15]:
for i in range(30):
    query_time = int(((datetime.datetime.now() - datetime.timedelta(days=i)) - datetime.datetime(1970,1,1)).total_seconds())
    #print query_time
    with con:
        cur.execute("INSERT INTO daily_temp(query_time) VALUES (?)", (query_time,))
     
    for city, loc in cities.items():
        api_call = url + api_key + '/' + loc + ',' + str(query_time)

        r = requests.get(api_call)
        max_temp = r.json()['daily']['data'][0]['temperatureMax']
        
        with con:
            cur.execute(
                "UPDATE daily_temp SET " + city + " = " + str(max_temp) + " " +
                "WHERE query_time = " + str(query_time)
            )